In [1]:
import sqlite3 as sql
import math
import catalog_utils as cu

In [2]:
def init(con, source):
    cur = con.cursor()
    target = source + '_p2'
    cur.execute('''CREATE TABLE '''+ target + ''' AS 
                   SELECT desig, 
                   m3_6, m4_5, mJ, mH, mKs, dm3_6, dm4_5, dmJ, dmH, dmKs, 
                   c3645, cHK, cJH, cK36
                   FROM '''
                   + source + ''' WHERE 
                   (m5_8 IS NULL OR m8_0 IS NULL) AND
                   (dmH < ? AND dmKs < ? AND (dmJ < ? OR mJ IS NULL)) AND
                   dm3_6 < ? AND
                   dm4_5 < ?''', [0.1, 0.1, 0.1, 0.2, 0.2]) 
                                #Relax the condition on dm3_6, dm4_5? not explicit in paper
    return target

In [3]:
#compute intrinsic H-K colors
def compute_icHK(con, source):
    cur = con.cursor()
    target = source + '_icHK'
    cur.execute('''CREATE TABLE '''+ target + ''' AS SELECT *,
                   CASE
                   WHEN (mJ IS NOT NULL) AND cHK > 0.58 * cJH - 0.21 
                     THEN 1.5 * cHK - 0.87 * cJH + 0.45
                   WHEN (mJ IS NOT NULL) AND cHK <= 0.58 * cJH - 0.21
                     THEN cHK - 0.58 * cJH + 0.35
                   WHEN mJ IS NULL AND cHK < -0.12 + 5.4 * c3645 
                     THEN 1.76 * c3645 - 0.32 * cHK + 0.176
                   WHEN mJ IS NULL AND cHK >= -0.12 + 5.4 * c3645 
                     THEN 0.2
                   END AS icH_K
                   FROM '''+ source)
    return target

In [4]:
def compute_icK36_ic3645(con, source):
    cur = con.cursor()
    target = source + '_icK36_ic3645'
    cur.execute('''CREATE TABLE '''+ target + ''' AS SELECT *, 
                   cK36 - (cHK - icH_K) * 0.671 AS icK_36,
                   c3645 - (cHK - icH_K) * 0.184 AS ic36_45 FROM '''+ source)
    return target

In [5]:
def extract_yso(con, source, applysigma=False):
    cur = con.cursor()
    target = source + '_yso'
    if applysigma == False:
        cur.execute('''CREATE TABLE '''+ target + ''' AS SELECT * FROM '''+ source + ''' WHERE
                   ic36_45 > 0.101 AND
                   icK_36 > 0 AND
                   icK_36 > -2.85714 * (ic36_45 - 0.101) + 0.5''' )
    
    else:
        cur.execute('''CREATE TABLE '''+ target + ''' AS SELECT * FROM '''+ source + ''' WHERE
                   ic36_45 - SQRT(SQUARE(dm3_6) + SQUARE(dm4_5)) > 0.101 AND
                   icK_36 - SQRT(SQUARE(dmKs) + SQUARE(dm3_6)) > 0 AND
                   icK_36 - SQRT(SQUARE(dmKs) + SQUARE(dm3_6)) > -2.85714 * (ic36_45 - SQRT(SQUARE(dm3_6) + SQUARE(dm4_5)) - 0.101) + 0.5''' )
    return target

In [6]:
def extract_1(con, source, cutoff361, applysigma=False):
    cur = con.cursor()
    target = source + '_1'
    if applysigma == False:
        cur.execute('''CREATE TABLE '''+ target +''' AS SELECT desig FROM '''+ source +''' WHERE
                   ic36_45 > -2.85714 * (ic36_45 - 0.401) + 1.7
                   AND
                   m3_6 < ?''', [cutoff361])
    else:
        cur.execute('''CREATE TABLE '''+ target +''' AS SELECT desig FROM '''+ source +''' WHERE
                   ic36_45 - SQRT(SQUARE(dm3_6) + SQUARE(dm4_5)) > 
                   -2.85714 * (ic36_45 - SQRT(SQUARE(dm3_6) + SQUARE(dm4_5)) - 0.401) + 1.7
                   AND
                   m3_6 < ?''', [cutoff361])
    return target

In [7]:
def extract_2(con, yso, c1, cutoff362):
    cur = con.cursor()
    target = yso+'_2'
    cur.execute('''CREATE TABLE '''+ target +''' AS SELECT desig FROM '''+ yso +''' WHERE 
                   m3_6 < ? 
                   AND
                   desig NOT IN (SELECT desig FROM '''+ c1 +''')''', [cutoff362])
    return target

In [8]:
def run_phase2(master, constr, cutoff361, cutoff362, applysigma=False):
    con = sql.connect(constr)
    
    #add math udf to sqlite
    cu.add_math(con)
    
    #initialize a working copy containing relevant data
    wc = init(con, master)
    cu.update_phase(con, master, wc, 2)
    
    #compute three intrinsic colors
    icHK = compute_icHK(con, wc)
    icK36_ic3645 = compute_icK36_ic3645(con, icHK)
    
    yso = extract_yso(con, icK36_ic3645, applysigma)
    
    c1 = extract_1(con, yso, cutoff361, applysigma)
    cu.update_type(con, master, c1, 1)
    
    c2 = extract_2(con, yso, c1, cutoff362)
    cu.update_type(con, master, c2, 2)
    
    return 0

In [9]:
run_phase2('gtw49', '/home/kecai/w49/w49test.db', 15.2, 15.2, 0)

0